
# AA Flight Delay EDA & Labeling (CSV Version)

This notebook:
- Loads `AA_FLIGHT_RECORD_FINAL_HUBS.csv`
- Computes delays and labels:
  - `dep_delay = DEP_TIME - CRS_DEP_TIME`
  - `arr_delay = ARR_TIME - CRS_ARR_TIME`
  - `delta_delay = arr_delay - dep_delay`
  - `delay_change_flag = 1 if |delta_delay| > threshold else 0`
- Cleans cancellations/diversions and trims extreme outliers
- Produces summary statistics and EDA plots
- Saves a labeled CSV for downstream modeling


In [ ]:

# === Config ===
CSV_PATH = "AA_FLIGHT_RECORD_FINAL_HUBS.csv"   # Update if your CSV is elsewhere
OUTPUT_DIR = "outputs"
PLOTS_SUBDIR = "plots"
LABELED_CSV_NAME = "aa_flights_labeled.csv"

DELTA_THRESHOLD_MIN = 10   # minutes for classification label |delta_delay|
MAX_ABS_DELAY_MIN = 600    # outlier cutoff in minutes (10 hours)
SAMPLE_SCATTER = 20000     # max points shown in scatter plot


In [ ]:

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Optional: pretty plots (can be commented out if seaborn not installed)
try:
    import seaborn as sns
    sns.set_style("whitegrid")
except Exception:
    pass

pd.options.display.max_columns = 120
print("pandas version:", pd.__version__)


In [ ]:

def hhmm_to_minutes(val):
    """Convert HHMM (e.g., 730) or HMM (e.g., 55) to minutes since midnight."""
    if pd.isna(val):
        return np.nan
    try:
        iv = int(val)
        h = iv // 100
        m = iv % 100
        if 0 <= h <= 48 and 0 <= m < 60:  # loose guard
            return h * 60 + m
    except Exception:
        pass
    return np.nan


def time_diff_minutes(actual_min, sched_min):
    """Compute (actual - scheduled) minutes with simple day-rollover heuristics."""
    if np.isnan(actual_min) or np.isnan(sched_min):
        return np.nan
    diff = actual_min - sched_min
    # Handle potential midnight crossovers:
    if diff < -720:   # >12h early -> assume actual is next day
        diff += 1440
    elif diff > 720:  # >12h late  -> assume scheduled is next day
        diff -= 1440
    return diff


def ensure_dir(path):
    os.makedirs(path, exist_ok=True)


In [ ]:

# === Load CSV ===
ensure_dir(OUTPUT_DIR)
plots_dir = os.path.join(OUTPUT_DIR, PLOTS_SUBDIR)
ensure_dir(plots_dir)

df = pd.read_csv(CSV_PATH)
df.columns = [c.strip() for c in df.columns]
print("Rows:", len(df))
df.head()


In [ ]:

# === Convert HHMM-like times to minutes since midnight ===
for col in ["CRS_DEP_TIME", "DEP_TIME", "CRS_ARR_TIME", "ARR_TIME"]:
    if col not in df.columns:
        raise ValueError(f"Missing required column: {col}")
    df[col + "_MIN"] = df[col].apply(hhmm_to_minutes)

# === Compute delays ===
df["dep_delay"]   = [time_diff_minutes(a, s) for a, s in zip(df["DEP_TIME_MIN"], df["CRS_DEP_TIME_MIN"])]
df["arr_delay"]   = [time_diff_minutes(a, s) for a, s in zip(df["ARR_TIME_MIN"], df["CRS_ARR_TIME_MIN"])]
df["delta_delay"] = df["arr_delay"] - df["dep_delay"]

# === Filter cancelled/diverted if present ===
for flag_col in ["CANCELLED", "DIVERTED"]:
    if flag_col in df.columns:
        df = df[(df[flag_col].isna()) | (df[flag_col] == 0)]

# === Drop rows missing core values ===
df = df.dropna(subset=["dep_delay", "arr_delay", "delta_delay"]).copy()

# === Trim extreme outliers ===
mask = (
    df["dep_delay"].abs() <= MAX_ABS_DELAY_MIN
) & (
    df["arr_delay"].abs() <= MAX_ABS_DELAY_MIN
) & (
    df["delta_delay"].abs() <= MAX_ABS_DELAY_MIN * 2
)
removed = (~mask).sum()
df = df[mask].copy()

print(f"Rows after cleaning: {len(df):,} (removed {removed:,} outliers)")

# === Classification label ===
df["delay_change_flag"] = (df["delta_delay"].abs() > DELTA_THRESHOLD_MIN).astype(int)

df[["dep_delay","arr_delay","delta_delay","delay_change_flag"]].head()


In [ ]:

# === Summary statistics ===
display(df[["dep_delay","arr_delay","delta_delay"]].describe().round(2))

print("Class balance (delay_change_flag):")
display(df["delay_change_flag"].value_counts(normalize=True).rename("share").round(3))


In [ ]:

# === Histograms ===
def hist_plot(series, title, fname, bins=60, xmin=-120, xmax=180):
    s = series.dropna()
    s = s[(s >= xmin) & (s <= xmax)]
    plt.figure(figsize=(7, 4.5))
    plt.hist(s.values, bins=bins)
    plt.title(title)
    plt.xlabel("Minutes")
    plt.ylabel("Count")
    plt.grid(True, linestyle="--", alpha=0.4)
    plt.tight_layout()
    plt.savefig(os.path.join(plots_dir, fname), dpi=150)
    plt.show()

hist_plot(df["dep_delay"],   "Departure Delay (min)",                "hist_dep_delay.png")
hist_plot(df["arr_delay"],   "Arrival Delay (min)",                  "hist_arr_delay.png")
hist_plot(df["delta_delay"], "Delta Delay (arr - dep) (min)",        "hist_delta_delay.png")


In [ ]:

# === Scatter: dep_delay vs arr_delay ===
sample = df.sample(min(SAMPLE_SCATTER, len(df)), random_state=42)
plt.figure(figsize=(6.5, 6.5))
plt.scatter(sample["dep_delay"], sample["arr_delay"], s=6, alpha=0.25)
# Reference line y=x
lims = [
    min(plt.xlim()[0], plt.ylim()[0]),
    max(plt.xlim()[1], plt.ylim()[1]),
]
plt.plot(lims, lims, linestyle="--", linewidth=1)
plt.xlim(lims)
plt.ylim(lims)
plt.title("Arrival Delay vs Departure Delay")
plt.xlabel("dep_delay (min)")
plt.ylabel("arr_delay (min)")
plt.grid(True, linestyle="--", alpha=0.4)
plt.tight_layout()
plt.savefig(os.path.join(plots_dir, "scatter_arr_vs_dep.png"), dpi=150)
plt.show()


In [ ]:

# === Destination-level summary (if DEST exists) ===
if "DEST" in df.columns:
    dest_summary = (
        df.groupby("DEST")
          .agg(
              n_flights=("delta_delay", "size"),
              delta_mean=("delta_delay", "mean"),
              pct_makeup=("delta_delay", lambda x: (x < 0).mean() * 100.0),
          )
          .sort_values("n_flights", ascending=False)
    )
    display(dest_summary.head(20))
    dest_summary.to_csv(os.path.join(OUTPUT_DIR, "airport_dest_summary.csv"))
    print("Saved →", os.path.join(OUTPUT_DIR, "airport_dest_summary.csv"))
else:
    print("Column 'DEST' not found; skipping destination summary.")


In [ ]:

# === Save labeled dataset ===
save_cols = [
    "FL_DATE" if "FL_DATE" in df.columns else None,
    "YEAR" if "YEAR" in df.columns else None,
    "MONTH" if "MONTH" in df.columns else None,
    "MKT_UNIQUE_CARRIER" if "MKT_UNIQUE_CARRIER" in df.columns else None,
    "OP_CARRIER_FL_NUM" if "OP_CARRIER_FL_NUM" in df.columns else None,
    "AIRCRAFT_TYPE" if "AIRCRAFT_TYPE" in df.columns else None,
    "DESCRIPTION" if "DESCRIPTION" in df.columns else None,
    "ORIGIN" if "ORIGIN" in df.columns else None,
    "DEST" if "DEST" in df.columns else None,
    "CRS_DEP_TIME", "DEP_TIME", "CRS_ARR_TIME", "ARR_TIME",
    "dep_delay", "arr_delay", "delta_delay", "delay_change_flag",
]
save_cols = [c for c in save_cols if c is not None and c in df.columns]
labeled_path = os.path.join(OUTPUT_DIR, LABELED_CSV_NAME)
df.to_csv(labeled_path, index=False, columns=save_cols)
print(f"Saved labeled dataset → {labeled_path} ({len(df):,} rows)")



---
### Next steps
- Join **METAR** weather snapshots (nearest-prior obs at ORIGIN/DEST) as additional features.
- Add route/airport priors (historical OTP by OD pair and hour-of-day).
- Train:
  - Classification: `abs(delta_delay) > DELTA_THRESHOLD_MIN`
  - Regression: `delta_delay` (predict magnitude and sign)
- Evaluate cost-sensitive metrics aligned with operations (e.g., false-hold vs missed-hold costs).
